# [Extracting features from scraped data](#extracting-features-from-scraped-data)

In [ ]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2

In [ ]:
from glob import glob
from pathlib import Path
from time import strftime

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

<a id="toc"></a>

## [Table of Contents](#table-of-contents)
0. [About](#load-data)
1. [User Inputs](#user-inputs)
2. [Load data](#load-data)
3. [Feature Engineering](#feature-engineering)
4. [Export to processed data file](#export-to-processed-data-file)

<a id="about"></a>

## 0. [About](#about)

In this notebook, we will extract new features from the data stored in the `*.csv` files in `data/*.csv`. The total number of listings retrieved for each city is shown below

In [ ]:
print("AUS", f"Number of pages of listings: {len(glob('data/AUS/*.csv'))}")
print("SEA", f"Number of pages of listings: {len(glob('data/SEA/*.csv'))}")

<a id="user-inputs"></a>

## 1. [User Inputs](#user-inputs)

For each city, we'll specify how many `*.csv` files of scraped car listings data to work with

In [ ]:
n_files_per_city = {
    "AUS": 29,
    "SEA": 22,
}

In [ ]:
file_id = "_".join([f"{k}{v}" for k, v in n_files_per_city.items()]) + "_2SEAzipcodes"

In [ ]:
number_files = sum(n_files_per_city.values())
number_files

We will also specify here the features that we will want to work with in our regression analysis. Missing data will be dropped based on these columns. In this way, missing values in other columns will not affect the subset of features we want to focus on here

In [ ]:
eng_nums = ["MPG", "tank_volume"]  # "Highway MPG" + "City MPG",
eng_cats = [
    "year",
    "make",
    "model",
    "trans_speed",
]

In [ ]:
nums = [
    "Mileage",
    "consumer_reviews",
    "seller_reviews",
    "price",
]
cats = [
    "Fuel Type",
    "Drivetrain",
    "seller_rating",
    "consumer_stars",
    "Comfort",
    "Performance",
    "Reliability",
]

<a id="load-data"></a>

## 2. [Load data](#load-data)

We'll begin by loading all `*.csv` files in `data/*.csv` into a single Pandas `DataFrame`.

In [ ]:
data_dir_path = Path().cwd() / "data"
print(data_dir_path)

In [ ]:
len(glob(str(data_dir_path / "*" / "*.csv")))

In [ ]:
df = pd.concat(
    [pd.read_csv(f, sep=",") for f in glob(str(data_dir_path / "*" / "*.csv"))],
    ignore_index=True,
)
print(df.shape)
df.head()

<a id="feature-engineering"></a>

## 3. [Feature Engineering](#feature-engineering)

We'll start by finding the number of missing values in the features and target (`price`) columns

In [ ]:
df[nums + cats + ["price"]].isna().sum().to_frame()

We can see there are missing values, but we won't drop these here. We'll put that step into a pre-processing pipeline that can be modified immediately before analysis.

We'll drop observations where the listing title is stored as the string `title` - this is a duplicate of the the header row and is a consequence of how the file was loaded

In [ ]:
df = df.loc[(~df["title"].str.contains("title"))]

We'll append new features to the loaded data, by extracting parts of existing features. The existing features will be retained in the data. Here, we will extract the tank volume from the engine information that was scraped. We'l do this with a [regular expression](https://en.wikipedia.org/wiki/Regular_expression) that extracts the numeric part of the `Engine` column, which will give the volume. We'll then convert this extracted string from a string into a `float`.

In [ ]:
df["tank_volume"] = (
    df["Engine"]
    .str.split("L", expand=True)[0]
    .str.strip()
    .str.extract("(\d\.\d)", expand=False)
    .astype(float)
)

We'll get the `make` and `model` from the `title`, since these are not explicitly provided in the listing and it seems intuitive that these categorical features would be helpful in predicting price of the car. Again, we'll use regular expression to help extract this information

In [ ]:
df["year"] = df["title"].str.extract("(\d+)").astype(int)
df["make_model"] = df["title"].str.replace("\d+ ", "")
df[["make", "model"]] = df["make_model"].str.split(" ", 1, expand=True)

We'll get the `City` and `State` from the `seller_address` column

In [ ]:
df[["City", "State"]] = df["seller_address"].str.extract(
    r"((?P<City>[A-Z][a-z]+),\s(?P<State>[A-Z]{2}))", expand=False
)[["City", "State"]]

As the transmission determines the proper quantity of power delivered to the wheels to allow the car to be driven at a specific speed, we'll extract the transmission speed from the `Transmission` column

In [ ]:
df["trans_speed"] = df["Transmission"].str.split("-Speed", expand=True)[0]

We'll calculate the interest, in dollars, to be paid

In [ ]:
df["interest"] = (df["per_month_min"].astype(float) * 60) - df["price"].astype(float)

Finally, we'll convert the `Mileage` column from a string into a numeric one

In [ ]:
df["Mileage"] = df.loc[df["Mileage"] != "Not provided"]["Mileage"].str.replace(",", "")

We'll also combine `Highway MPG` and `City MPG` into a single feature named `MPG`

In [ ]:
df["MPG"] = df["Highway MPG"] + df["City MPG"]

In [ ]:
# df[nums + cats + eng_nums + eng_cats + ["price"]].isnull().sum()

In [ ]:
# df[nums + cats + eng_nums + eng_cats + ["price"]].dropna(how='any').isnull().sum()

Finally, we'll convert the `price` column from a string into a `float`

In [ ]:
df["price"] = df["price"].astype(float)

In [ ]:
# df2 = df[nums + cats + eng_nums + eng_cats + ["price"]].copy()
# df2.dropna(how='any', inplace=True)

In [ ]:
# df2[nums + cats + eng_nums + eng_cats + ["price"]].isnull().sum()

In [ ]:
# df2[nums + cats + eng_cats + eng_nums].dropna(how="any").shape[0]

In [ ]:
# df2[nums + cats + eng_cats + eng_nums].shape[0]

In [ ]:
# df2[nums + cats].dropna(how="any").shape[0]

In [ ]:
# new_cols = [
#     "Engine",
#     "tank_volume",
#     "year",
#     "make",
#     "model",
#     "seller_address",
#     "City",
#     "State",
#     "MPG",
#     "Mileage",
#     "trans_speed",
#     "interest"
# ]
# df[new_cols].head()

<a id="export-to-processed-data-file"></a>

## 4. [Export to processed data file](#export-to-processed-data-file)

Finally, we'll export the processed data to a file

In [ ]:
df.to_csv(
    str(data_dir_path / f"processed_data__{file_id}_{strftime('%Y%m%d_%H%M%S')}.csv"),
    index=False,
)